In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pyautogui

import csv
import json
import time

from tqdm.notebook import tqdm

class StockVid(object):
    def __init__(self, stock, date, views):
        self.stock = stock
        self.date = date
        self.views = views
    
    def jsonEnc(self):
      return {'stock': self.stock, 'date': self.date, 'views': self.views}

def jsonDefEncoder(obj):
    if hasattr(obj, 'jsonEnc'):
        return obj.jsonEnc()
    else: #some default behavior
        return obj.__dict__

    
class youtubeScraper:

    def get_vids(self, stock):
        
        stockTickers = {}
        stockTickers[stock] = {}
        relevantVids = []
        
        driver = webdriver.Chrome('C:/WebDriver/chromedriver')
        driver.get("https://www.youtube.com/results?search_query=" + stock)
       
    
        ## Begin Scrolling ##
        start = time.perf_counter()
        print("Phase[1/2]--Scrolling", end=' ')
        while(True):
            try:
                driver.find_element_by_xpath("//*[@id='message'][text()='No more results']")
                break
            except:
                pyautogui.press('pagedown')
                continue
             
            break
            
        end = time.perf_counter()
        p1_time = divmod(int(end-start), 60)
        print("[{:02}:{:02}]".format(p1_time[0], p1_time[1]))
        
            
        ## Begin Scraping ##
        all_vids = driver.find_elements_by_xpath("//div[@ID='contents']/ytd-video-renderer")
        
        for i in tqdm(range(len(all_vids)), desc = 'Phase [2/2] -- Scraping'):
            vid_views = all_vids[i].find_element_by_xpath(".//*[@id='metadata-line']/span[1]").text
            
            try:
                vid_date = all_vids[i].find_element_by_xpath(".//*[@id='metadata-line']/span[2]").text
            except:
                vid_date = "live"
                
            stockTickers[stock][i] = StockVid(stock, vid_date, vid_views) 
            
        driver.close()
        
        ## Begin Formatting Results ##
        if(len(stockTickers[stock]) > 0):
            for post in stockTickers[stock]:
                relevantVids.append(stockTickers[stock][post])
        json_object = json.dumps(relevantVids, default=jsonDefEncoder, indent = 4)
        
        print(json_object) 
        
        
if __name__ == "__main__":
    youtubeScraper().get_vids('safemoon')

In [ ]:
## Initializing a connection to reddit_db

import mysql.connector

cnx = mysql.connector.connect(
    user='root',
    password='chalkHorseMountain',
    host = 'localhost',
    database='redditScraper'
    )

mycursor = cnx.cursor()

In [ ]:
## Creating youtube_<stock>_db
mycursor.execute("""
    CREATE TABLE youtube_""" + stock + """ (
        post_id INT, num_views INT, date_posted DATE
    );
    """)

In [ ]:
def table_exists(tbl_name):
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = DATABASE()
        AND table_name = \"""" + tbl_name + """\";
    """)
    
    if mycursor.fetchone()[0] == 1:
        return True
    return False